This program is free software; you can redistribute it and/or modify it under the terms of the GNU General Public License, version 2, as published by the Free Software Foundation.

This program is distributed in the hope that it will be useful, but WITHOUT ANY WARRANTY; without even the implied warranty of MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE. See the GNU General Public License for more details.

Author: Alexander Young and Nicolas Flament

### This notebook takes time-dependent hypsometry curves and modifies them for dyanmic topography. The present day ocean basin water volume calculated from the curve at time = 0 Ma is then modified for relative adjustments to submarine sediment and OLIP volumes, trench volume and deep water flux volume. The adjusted volume is then used to flood the hypsometry curve and sea level is calculated.

##### Input:
time-dependent trench volumes produced from 1_SeafloorBathymetry_from_AgeGrids_and_TrenchVolume.ipynb e.g. C1TT_sz_300_km_normalised_volume_trench_age.dat

time-dependent sediment thickness produced from 2-Sediment_thickness_model.ipynb e.g. SedModelFinal_0.0-580.0Ma_500km_Clipped5km.csv

time-dependent OLIP volumes produced from 3-Volumes_of_submarine_LIPs.ipynb e.g. Model_volumes_of_submarine_LIPs.csv

time-dependent deep water flux produced from 5-Calculate_DeepWaterFlux.ipynb e.g. NNRPlateModel_RD_DeepWaterFlux_AllTimes.csv

time-dependent dynamic topographys produced from 7-Calc_Air_Loaded_DT.ipynb e.g. C1_mean_TopoCheck_ocean_age_grid_AL.dat

time-dependent hypsometry models produced from 6-Model_timedependent_AL_hypsometry.ipynb e.g. hypsoModel_HypsoModelFinal_0Ma_fOA_fS_KRCC.csv

##### Output:
time-dependent sea level file e.g. C1_0-500Ma_SeaLevelFinal_sedClipped_fOA_fS_KRCC.csv


In [14]:
from __future__ import print_function
import math
import os
import os.path
import sys
sys.path.insert(1, '/Users/ajy321/pygplates_rev18_python27_MacOS64')
import pygplates
# Add directory containing the 'ptt' module (Plate Tectonic Tools) to the Python path.
import pandas as pd
import math
import matplotlib.pyplot as plt
import scipy.interpolate as spi
import numpy as np
from matplotlib import cm
from functools import reduce
from sklearn.preprocessing import minmax_scale
from numpy import ones,vstack
from numpy.linalg import lstsq
from sklearn import preprocessing

# Define the time range.
# The reconstruction time range (topologies resolved to these times).
# Also used to get paleo raster filenames based on 'raster_filename_base'.
min_time = 0
max_time = 600
time_step = 100

# define geodynamic model
model='gld428'

# Toggle dynamicOceanArea to call model for dynamic or static ocean basin area.
# 1 dynamic, 0 fixed
dynamicOceanArea = 0.

# Toggle dynamicSlope to to call model for dynamic or static coastal gradient.
# 1 dynamic, 0 fixed
dynamicSlope = 0.

# Toggle age_depth_model to call model for half space cooling or simple plate.
# 0 simple plate, 1 half space cooling, 2 complex
age_depth_model = 0.

# Toggle dynamic topography
# 1 No Dt, 0 include Dt
DT = 0.

# write program to search for number and return index
def find_nearest(array, value):
        array = np.asarray(array)
        idx = (np.abs(array - value)).argmin()
        return array[idx]

resample_list=np.arange(0.0, 1.0, 1e-6)

In [15]:
# -- load Hypsometric curve for etopo1 produced by Eakins et al., 2012
# -- set output file names 
etopoHistFile="/Users/ajy321/PhD_work/SeaLevel/USYDmodel/DependentInputs/etopo1_hyp_curve.dat"
etopoHist = pd.read_csv(etopoHistFile, skiprows=1, names=['ELEVATION', 'Area_per'],
                        delim_whitespace=True, usecols=['ELEVATION', 'Area_per'])

In [13]:
# Load in DT model
# Path to modeled dynamic topography
dtDir = '/Users/ajy321/PhD_work/SeaLevel/NF_workflows/Average_dynamic_elevation/gld428/Topo'
contDTdf = pd.read_csv(dtDir+'/%s_mean_TopoCheck_conts_age_grid_AL.dat' %(model),
                       delimiter = " ", header=None, names=['Age', 'contDT'])
oceanicDTdf = pd.read_csv(dtDir+'/%s_mean_TopoCheck_ocean_age_grid_AL.dat'%(model),
                          delimiter = " ", header=None, names=['Age', 'oceanicDT'])
oceanAreadf = pd.read_csv('/Users/ajy321/PhD_work/SeaLevel/NF_workflows/Average_dynamic_elevation/gld428/Topo/gld428_OceanArea.dat',
                          delimiter = " ", header=None, names=['Age', 'Area_m'])

# Designate present day continental and oceanic DT
contpd = float(contDTdf['contDT'].iloc[-1])
oceanpd = float(oceanicDTdf['oceanicDT'].iloc[-1])

# Calibrate continental and oceanic DT wrt to present day
contDTdf ['contDT'] -= contpd
oceanicDTdf ['oceanicDT'] -= oceanpd

# make and empty list to populate new sea levels to
target_vol_list=[]
result_vol_list=[]
SL_elev_list=[]
Vsed_list=[]
Vlip_list=[]
deepH20_list=[]
meanDepth = []
midDepth = []
contFlooding = []
baseShelf = []
dt_list = []

# Iterate over time steps
reconstruction_times = np.arange(min_time, max_time, time_step)
for reconstruction_time in reconstruction_times:
    print ("Age is: ",round(reconstruction_time, 1), "Ma")
        
    #Define cooling model
    if dynamicOceanArea == 1.:
        OAname = 'dOA'
    else:
        OAname = 'fOA'

    if dynamicSlope == 1.:
        Sname = 'dS'
    else:
        Sname = 'fS'
        
    if age_depth_model == 1.:
        coolingModel = 'HSCk'
        dmax = -1656. # this is a little bit greater than Zr to ensure the clip occurs beyond the continental shelf
    elif age_depth_model == 0.:
        coolingModel = 'Pk'
        dmax = -2216. # this is a little bit greater than Zr to ensure the clip occurs beyond the continental shelf
    else:
        coolingModel = 'KRCC'
        dmax = -2500. # this is a little bit greater than Zr to ensure the clip occurs beyond the continental shelf
        
    # Load air loaded hypsometry curve
    hypsoFile="/Users/ajy321/PhD_work/SeaLevel/Hypsometry/ModelAncientContHypso/gld428_0-560Ma_ModeledHypsometry_AL_fOA_fS_%s/gld428_HypsoModelFinal_%sMa_fOA_fS_%s.csv" %(coolingModel, reconstruction_time, coolingModel)
    hypsodf = pd.read_csv(hypsoFile, header=None, names=['Area_per', 'ELEVATION'])
    
    #increase spatial resolution of curve
    resampledf = pd.DataFrame({'Area_per':resample_list})
    hypso_hidef = pd.merge(resampledf, hypsodf, on='Area_per', how='outer')
    hypso_hidef = hypso_hidef.sort_values(by='Area_per', ascending=True)
    hypso_hidef = hypso_hidef.interpolate(method='linear', axis=0, limit=None, limit_direction='both').ffill().bfill()
    hypso_hidef['Area_m'] = hypso_hidef['Area_per'] * 5.1e14
    
    # set model dependent Acc (cum area at ridge depth)
    Acc = 0.407837 # present day ocean area
    
    # set model dependent Ash (cum area at base of the continental shelf; -250m)
    Zsh=-250.
    index_loc_Ash = abs(hypso_hidef['ELEVATION'] - Zsh).idxmin()
    Ash = float(hypso_hidef['Area_per'].iloc[index_loc_Ash])
    
    # set model dependent sea level area. 
    Zsl = 0.
    index_loc_SL = abs(hypso_hidef['ELEVATION'] - Zsl).idxmin()
    SL = float(hypso_hidef['Area_per'].iloc[index_loc_SL])
    
    # set model dependent Af (cum area at top of the continental shelf; 200m)
    Ztop=200.
    index_loc_Af = abs(hypso_hidef['ELEVATION'] - Ztop).idxmin()
    Af = float(hypso_hidef['Area_per'].iloc[index_loc_Af])
    
    # calculate volume above ridge from !*! AIR LOADED !*! bathy grid
    # set variables to unload bathy
    rhom = 3340. # mantle density
    rhow = 1030. # water density
    rhoa = 1. # air density
    waterLoad = (rhoa - rhom) / (rhow - rhom)
    waterUnload = (1/waterLoad)
    iso = ((rhom - rhow)/rhom)
    
    # Earth Area
    AE = 5.1e14
    
    # Extract oceanic area
    OArea=(oceanAreadf.loc[oceanAreadf['Age'] == reconstruction_time, 'Area_m'].item())
#     print ('ocean area is', OArea)
    CArea = (AE - OArea)
    CAreaWeight = CArea/OArea

    # Add DT if required
    if DT == 1:
        hypso_hidef['ELEVATION_DT'] = hypso_hidef['ELEVATION']
        oceanicDT = 0.0
        contDT = 0.0
    else:
        # add air loaded DT
        oceanicDT=(oceanicDTdf.loc[oceanicDTdf['Age'] == reconstruction_time, 'oceanicDT'].item())
        contDT=(contDTdf.loc[contDTdf['Age'] == reconstruction_time, 'contDT'].item())
        DT_t = oceanicDT - (contDT * CAreaWeight)
        hypso_hidef.loc[hypso_hidef.Area_per >= Acc, 'ELEVATION_DT'] = hypso_hidef.ELEVATION + (DT_t)
        hypso_hidef.loc[hypso_hidef.Area_per < Acc, 'ELEVATION_DT'] = hypso_hidef.ELEVATION
        dt_list.append(DT_t)
        
    # split the hypso curve at the top (4000m) to calculate max volume, at the and base (-4000m) to calculate minimum volume
    # and at SL (0m) to calculate present day volume
    hypso_min = hypso_hidef.drop(hypso_hidef[hypso_hidef.ELEVATION_DT > -1000.].index)
    hypso_max = hypso_hidef.drop(hypso_hidef[hypso_hidef.ELEVATION_DT > 11000.].index)
    hypso_target = hypso_hidef.drop(hypso_hidef[hypso_hidef.ELEVATION_DT > Zsl].index)
    
    # make all values negative so we can waterload
    hypso_min["all_negative"] = (hypso_min["ELEVATION_DT"] - hypso_min["ELEVATION_DT"].iloc[0])
    hypso_max["all_negative"] = (hypso_max["ELEVATION_DT"] - hypso_max["ELEVATION_DT"].iloc[0])
    hypso_target["all_negative"] = (hypso_target["ELEVATION_DT"] - hypso_target["ELEVATION_DT"].iloc[0])

    # now water load
    hypso_min["ELEVATION_WL"] = hypso_min["all_negative"] * waterLoad 
    hypso_max["ELEVATION_WL"] = hypso_max["all_negative"] * waterLoad 
    hypso_target["ELEVATION_WL"] = hypso_target["all_negative"] * waterLoad

    # make all values positive so we can calculate area above the curve
    hypso_min["all_positive"] = hypso_min["ELEVATION_WL"] - float(hypso_min['ELEVATION_WL'].iloc[-1])
    hypso_max["all_positive"] = hypso_max["ELEVATION_WL"] - float(hypso_max['ELEVATION_WL'].iloc[-1])
    hypso_target["all_positive"] = hypso_target["ELEVATION_WL"] - float(hypso_target['ELEVATION_WL'].iloc[-1])

    # add datum column to dataframe so we can calculate area above the curve
    hypso_min['Zsh_min']= (0 + float(hypso_min['all_positive'].iloc[0]))
    hypso_max['Ztop_max']= (0 + float(hypso_max['all_positive'].iloc[0]))
    hypso_target['Z_sL']= (0 + float(hypso_target['all_positive'].iloc[0]))
    
    # find difference between datum and hypso curve
    diff_min = hypso_min['Zsh_min'].values - hypso_min['all_positive'].values
    diff_max = hypso_max['Ztop_max'].values - hypso_max['all_positive'].values
    diff_target = hypso_target['Z_sL'].values - hypso_target['all_positive'].values
    
    # caluculate volume betwen the datum and the hypso curve
    volume_min = np.trapz(diff_min, hypso_min['Area_m'].values)
    volume_max = np.trapz(diff_max, hypso_max['Area_m'].values)
    volume_target = np.trapz(diff_target, hypso_target['Area_m'].values)
    target_vol_list.append(volume_target)

    # Load in sediment thickness model
    sedFile="/Users/ajy321/PhD_work/SeaLevel/SedimentThickness/SedModelFinal_0-580Ma_400km.csv"
    seddf = pd.read_csv(sedFile)
    # calculate the compoent of volume displaced by the sediments
    seddf["seds_wrt_pd"] = seddf["mean_thicknes"] - float(seddf['mean_thicknes'].iloc[0])
    Vsed = ((1. - Acc) * AE) * (seddf.loc[seddf['Age'] == reconstruction_time, 'seds_wrt_pd'].item() * 1000.)
    Vsed_list.append(Vsed)
    
    # Load in LIP volume model
    LIPFile="/Users/ajy321/PhD_work/SeaLevel/USYDmodel/OLIPModel/Model_volumes_of_submarine_LIPs.csv"
    LIPdf = pd.read_csv(LIPFile)
    # calculate the compoent of volume displaced by the LIPs
    LIPdf["Lip_wrt_pd"] = LIPdf["50per"] - float(LIPdf['50per'].iloc[0])
    Vlip = (LIPdf.loc[LIPdf['Age'] == reconstruction_time, 'Lip_wrt_pd'].item())
    Vlip_list.append(Vlip)
    
    # Load deep water recycling model
    deepwaterFile="/Users/ajy321/PhD_work/SeaLevel/SeaLevel_MS/PythonNotebooks/DeepwaterCycling/NNRPlateModel_RD_DeepWaterFlux_AllTimes.csv"
    deepwaterdf = pd.read_csv(deepwaterFile)
    deepwaterdf["volume_wrt_pd"] = deepwaterdf["volume_m3"] - float(deepwaterdf['volume_m3'].iloc[0])
    # set volume of water contributed by the deep water cycle
    Vdeepwater = (deepwaterdf.loc[deepwaterdf['Time'] == reconstruction_time, 'volume_wrt_pd'].item())
    deepH20_list.append(Vdeepwater)
    
    # Load volume of trenches
    trenchFile="/Users/ajy321/PhD_work/SeaLevel/BathymetryModeling/gld428_Volumes/gld428TT_sz_300_km_normalised_volume_trench_age.dat"
    trenchdf = pd.read_csv(trenchFile, names=['Time', 'volume_m3'], delim_whitespace=True)
    #calibrate trench volumes with respect to present day
    trenchdf["volume_wrt_pd"] = trenchdf["volume_m3"] - float(trenchdf['volume_m3'].iloc[-1])
    # set volume of water displaced by oceanic trenches
    Vtrench = (trenchdf.loc[trenchdf['Time'] == reconstruction_time, 'volume_wrt_pd'].item())
    
    # Load Stap et al. 2017 ice-volume effects on SL for the last 38 myr
    StapFile="/Users/ajy321/PhD_work/SeaLevel/USYDmodel/DeBoer-et-al-2010_Glaciology.csv"
    Stapdf = pd.read_csv(StapFile)
    #set volume of water sotred as glacial ice
    Vice = ((Stapdf.loc[Stapdf['Time'] == reconstruction_time, 'ice_SLwrt'].item())) * iso
  
    # Set minimum volume
    Vmin = (volume_min)

    # Set maximum volume
    Vmax = (volume_max)
     
    # Time to flood the curve    
    if reconstruction_time == 0.:
        x = volume_target
    
    # modify present day volume with contribution volumes
    # VoTarget = x + Vsed + Vlip - Vtrench + Vdeepwater
    VoTarget = x + Vsed + Vlip - Vtrench + Vdeepwater
    
    lim=VoTarget/1e4
    
    # sometimes the iteration process does not converge; adding an aribtrary DC shift (_alpha)
    # usually resolves this
    _alpha=0.
    
    # iterative approach will not exactly locate VoTrue,
    # so provide a constrained solution range here
    # using VoMax and VoMin
    VoMax=VoTarget+lim
    VoMin=VoTarget-lim
    
    # iteration counters
    # first iteration
    it=0
    
    # maximum number of iterations
    iterations=100
    
    # minimum topography value is used as a contour
    ## the iteration process needs two values to start off, the further apart the better
    ## cont1 and cont2 are the minimum and maximum bounds for sea level (deepest abyss and highest mountains)
    cont1=(float(hypso_min['all_positive'].iloc[0]))

    # maximum topography value is used as a contour
    cont2=(float(hypso_max['all_positive'].iloc[0]))
    
    # volume below minimum contour level
    v1_min=Vmin
    
    # volume contained within maximum contour
    v2_max=Vmax
    
    contf=0.0
    
    ## contf will be the returned value. Our first guess is 0.
    while it < iterations and contf == 0.0:
        
        ## ensuring that contf is within bounds throughout iterations
        if v1_min < VoMax and v1_min > VoMin:
            contf=cont1 + Vice
        elif v2_max < VoMax and v2_max > VoMin:
            contf=cont2 + Vice
        else:
            cont3=cont2+(1.0+_alpha)*(cont1-cont2)*(VoTarget-v2_max)/(v1_min-v2_max)
            if cont3 == cont2:
                contf=cont2 + Vice
        ## ensuring that contf is within bounds throughout iterations
            else:
                if cont3 > cont2:
                    cont3=cont2
                elif cont3 < cont1:
                    cont3=cont1
        ## Calculating the volume of oceans above cont3
        # here we take the preesnt day water volume, modified for past contributions and use it to inundate 
        #the time dependent hypsometry
                cont3 = (cont3 + Vice)
                hypso_z3 = hypso_max.drop(hypso_max[hypso_max.all_positive > (cont3)].index)
                hypso_z3['z3_datum']= (0 + float(hypso_z3['all_positive'].iloc[0]))
                diff_z3 = hypso_z3['z3_datum'].values - hypso_z3['all_positive'].values
                v3 = np.trapz(diff_z3, hypso_z3['Area_m'].values)
                
        ## Checking if that volume v3 meets convergence criterion
            if v3 < VoMax and v3 > VoMin:
                contf=(float(hypso_z3['all_positive'].iloc[0])) + Vice

        ## If not updating either upper bound or lower bound
            elif v3 > VoMax:
                v2_max=v3
                cont2=cont3
            elif v3 < VoMin:
                v1_min=v3
                cont1=cont3
                
            it+=1
            if it == iterations:

                _alpha=_alpha+0.2
                contf=(float(hypso_z3['all_positive'].iloc[0])) + Vice
                it=0
            if _alpha > 1.8:
                sys.exit()
    
    # find present day SL to calibrate curve with respect to present
    if reconstruction_time == 0.:
        SL_zero = ((hypso_z3["ELEVATION"].iloc[0]))
    SL_t = ((hypso_z3["ELEVATION_DT"].iloc[0] - SL_zero))    
    result_vol_list.append(v3)
    SL_elev_list.append(int(SL_t))
    print ("Sea level at", reconstruction_time,"Ma is", (int(SL_t)))

    # Create an empty dataframe to export hypso results to
    export = hypso_max
    export.loc[export.ELEVATION <= SL_t, 'hypso_WL'] = ((export['ELEVATION'] - SL_t) * waterLoad) + SL_t
    export.loc[export.ELEVATION > SL_t, 'hypso_WL'] = export['ELEVATION']
    export.loc[export.ELEVATION_DT <= SL_t, 'FinalELEVATION'] = ((export['ELEVATION_DT'] - SL_t) * waterLoad) + SL_t
    export.loc[export.ELEVATION_DT > SL_t, 'FinalELEVATION'] = export['ELEVATION_DT']
#     export.to_csv('./exportHypso_%s.csv' %(reconstruction_time))

    # calculate flooded continetnal area - used for plotting.
    array = export.as_matrix(columns=export.columns[1:2])
    SL = np.asscalar(find_nearest(array, SL_t))
    index_loc_0m = export.loc[export['ELEVATION'] == SL].index.tolist()
    contFLOOD = float(export['Area_per'].iloc[index_loc_0m])
    # append to lists
    contFlooding.append(contFLOOD)
    baseShelf.append(Acc)

    VoTarget = 0.
    
SLelevdf = pd.DataFrame({'Age[Ma]':reconstruction_times, 'SL_elevation[m]':SL_elev_list,
                        "ocean_basin_volume":target_vol_list,
                        "pd_water_volume":result_vol_list, "baseShelf":baseShelf,
                        "contFlooding":contFlooding,})

SLelevdf['contFlooding_m^2'] = (SLelevdf['baseShelf'] - SLelevdf['contFlooding']) * AE
SLelevdf.to_csv("%s_%s-%sMa_SeaLevelFinal_Basecase_%s_%s_%s.csv" %(model, min_time, (max_time-time_step), OAname, Sname, coolingModel))




Age is:  0.0 Ma
Sea level at 0 Ma is 0


/Users/ajy321/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:308: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.


Age is:  20.0 Ma
Sea level at 20 Ma is 39
Age is:  40.0 Ma
Sea level at 40 Ma is 57
Age is:  60.0 Ma
Sea level at 60 Ma is 65
Age is:  80.0 Ma
Sea level at 80 Ma is 94
Age is:  100.0 Ma
Sea level at 100 Ma is 106
Age is:  120.0 Ma
Sea level at 120 Ma is 118
Age is:  140.0 Ma
Sea level at 140 Ma is 65
Age is:  160.0 Ma
Sea level at 160 Ma is -38
Age is:  180.0 Ma
Sea level at 180 Ma is -6
Age is:  200.0 Ma
Sea level at 200 Ma is 1
Age is:  220.0 Ma
Sea level at 220 Ma is 0
Age is:  240.0 Ma
Sea level at 240 Ma is 0
Age is:  260.0 Ma
Sea level at 260 Ma is 160
Age is:  280.0 Ma
Sea level at 280 Ma is 174
Age is:  300.0 Ma
Sea level at 300 Ma is 302
Age is:  320.0 Ma
Sea level at 320 Ma is 380
Age is:  340.0 Ma
Sea level at 340 Ma is 385
Age is:  360.0 Ma
Sea level at 360 Ma is 458
Age is:  380.0 Ma
Sea level at 380 Ma is 453
Age is:  400.0 Ma
Sea level at 400 Ma is 445
Age is:  420.0 Ma
Sea level at 420 Ma is 204
Age is:  440.0 Ma
Sea level at 440 Ma is 245
Age is:  460.0 Ma
Sea level at